# Naive Bayes Formulation:

Predicting survival from titanic crash

In [1]:
import pandas as pd

df = pd.read_csv("data/titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [2]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [3]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [4]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

## One Hot Encoding

In [5]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [6]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [7]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [8]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [9]:
inputs.isna().sum()

Pclass      0
Age       177
Fare        0
female      0
dtype: int64

## Patching Missing Values

In [10]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [12]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.7873134328358209

In [13]:
X_test[0:10]

,Pclass,Age,Fare,female
325,1,36.000000,135.6333,1
439,2,31.000000,10.5000,0
526,2,50.000000,10.5000,1
664,3,20.000000,7.9250,0
593,3,29.699118,7.7500,1
600,2,24.000000,27.0000,1
597,3,49.000000,0.0000,0
754,2,48.000000,65.0000,1
528,3,39.000000,7.9250,0
8,3,27.000000,11.1333,1


In [14]:
y_test[0:10]

325    1
439    0
526    1
664    1
593    0
600    1
597    0
754    1
528    0
8      1
Name: Survived, dtype: int64

In [15]:
model.predict(X_test[0:10])

array([1, 0, 1, 0, 1, 1, 0, 1, 0, 1], dtype=int64)

In [16]:
model.predict_proba(X_test[:10])

array([[7.80669637e-04, 9.99219330e-01],
       [9.24995066e-01, 7.50049336e-02],
       [2.29211341e-01, 7.70788659e-01],
       [9.59388203e-01, 4.06117970e-02],
       [3.93649936e-01, 6.06350064e-01],
       [2.17084655e-01, 7.82915345e-01],
       [9.60918092e-01, 3.90819080e-02],
       [1.21641260e-01, 8.78358740e-01],
       [9.64632087e-01, 3.53679129e-02],
       [3.91781609e-01, 6.08218391e-01]])

## Cross Validation

In [17]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.8       , 0.768     , 0.704     , 0.80645161, 0.76612903])